# verilog

Challenge problem:
barrel shift multiplier performs multiplication


# fast


In [10]:
import z3
x = z3.Int('x')
decl = (x + x).decl()
z = x + x
ctx = z3.main_ctx()

In [20]:
%%timeit
decl(x, x)

24.2 μs ± 320 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [21]:
%%timeit
x + x

22.6 μs ± 360 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
z3.main_ctx().ctx

In [28]:
x.as_ast()

In [ ]:
#z3.Z3_mk_app(decl.ref)


In [7]:
def mk_bin(f, a, b):
    args = (z3.Ast * 2)()
    args[0] = a.as_ast()
    args[1] = b.as_ast()
    return f(a.ctx.ref(), 2, args)

In [23]:
z3.z3_debug()

False

In [25]:
%%timeit
decl(x,x)

21.6 μs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
z3.z3.Z3_DEBUG = False
#z3.Z3_DEBUG

In [11]:
%%timeit
z3.ArithRef(mk_bin(z3.Z3_mk_add, x, x), ctx)

2.25 μs ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [24]:
%%timeit
x + x

19.7 μs ± 50.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [32]:
(z3.Ast * 2)()

In [50]:
z = x + x

In [4]:
%%timeit
z.arg(1)

5.46 μs ± 28.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
%%timeit
z.eq(x)

691 ns ± 3.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
%%timeit
z3.Z3_mk_add(z3.main_ctx().ctx, 2, x.as_ast(), x.as_ast())


AttributeError: 'Ast' object has no attribute 'f'

In [49]:
%%prun -s tottime
for _ in range(10000):
    decl(x, x)

         3040477 function calls (3010515 primitive calls) in 0.718 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   310000    0.106    0.000    0.118    0.000 z3core.py:1576(Check)
   120000    0.082    0.000    0.139    0.000 z3core.py:2873(Z3_sort_to_ast)
   9998/0    0.037    0.000    0.000          z3.py:855(__call__)
    50000    0.034    0.000    0.058    0.000 z3core.py:1646(Z3_inc_ref)
   210000    0.034    0.000    0.044    0.000 z3.py:410(ctx_ref)
   120000    0.034    0.000    0.196    0.000 z3.py:576(as_ast)
    50000    0.033    0.000    0.136    0.000 z3.py:360(__del__)
    50000    0.032    0.000    0.163    0.000 z3.py:355(__init__)
   670000    0.028    0.000    0.028    0.000 z3types.py:24(from_param)
    50000    0.028    0.000    0.032    0.000 z3core.py:1650(Z3_dec_ref)
    40000    0.026    0.000    0.045    0.000 z3core.py:3118(Z3_get_ast_id)
    30000    0.021    0.000    0.035    0.000 z3core.py:3

In [28]:
Foo = z3.DeclareSort("Foo")
bar = z3.Function("bar", Foo, Foo)
biz = z3.Const("biz", Foo)

In [30]:
%%timeit
bar(biz)

8.65 μs ± 49 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


It makes sorts way too many times.
_coerce


In [31]:
%%timeit 
x = 7
if x == 0:
    y = 4
elif x == 1:
    y = 5
elif x == 2:
    y = 6
elif x == 3:
    y = 7
elif x == 4:
    y = 8
elif x == 5:
    y = 9
elif x == 6:
    y = 10
elif x == 7:    
    y = 11
elif x == 8:
    y = 12

42.6 ns ± 0.21 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [29]:
%%prun -s cumtime
for _ in range(10000):
    bar(biz)

         1110682 function calls (1090670 primitive calls) in 0.277 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.277    0.277 {built-in method builtins.exec}
        1    0.005    0.005    0.277    0.277 <string>:1(<module>)
    10000    0.022    0.000    0.261    0.000 z3.py:855(__call__)
    10000    0.005    0.000    0.084    0.000 z3.py:792(domain)
    10000    0.012    0.000    0.069    0.000 z3.py:1176(_to_expr_ref)
    10000    0.007    0.000    0.061    0.000 z3.py:674(_to_sort_ref)
    20000    0.014    0.000    0.058    0.000 z3.py:355(__init__)
    20000    0.014    0.000    0.047    0.000 z3.py:360(__del__)
   100000    0.035    0.000    0.039    0.000 z3core.py:1576(Check)
    20000    0.006    0.000    0.035    0.000 z3.py:576(as_ast)
    10000    0.008    0.000    0.027    0.000 z3.py:152(_get_args)
    20000    0.014    0.000    0.024    0.000 z3core.py:2873(Z3_sort_to_as

In [27]:
%%prun -s cumtime
for _ in range(10000):
    x + x

         2810560 function calls (2810554 primitive calls) in 0.656 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    0.656    0.656 {built-in method builtins.exec}
      2/1    0.004    0.002    0.652    0.652 <string>:1(<module>)
    10000    0.009    0.000    0.637    0.000 z3.py:2499(__add__)
    10000    0.018    0.000    0.547    0.000 z3.py:1245(_coerce_exprs)
    20000    0.012    0.000    0.244    0.000 z3.py:2407(cast)
   140000    0.040    0.000    0.233    0.000 z3.py:576(as_ast)
    40000    0.027    0.000    0.233    0.000 z3.py:2464(sort)
    20000    0.008    0.000    0.184    0.000 z3.py:1226(_coerce_expr_merge)
    50000    0.034    0.000    0.168    0.000 z3.py:355(__init__)
    30000    0.022    0.000    0.167    0.000 z3.py:414(eq)
   140000    0.096    0.000    0.165    0.000 z3core.py:2873(Z3_sort_to_ast)
    50000    0.034    0.000    0.141    0.000 z3.py:360(__del__)
  

# effect handlers
Hmm.
do(Read, v)
do(Write, a)

How to keep the continuation straigth?

bad idea prob: use sexp.



# llvmlit jit
Talk 
https://github.com/sdiehl/numpile/blob/master/numpile.py

Sortdispatch reshitry of sort to compilation
singledispatch registry of 

emit mlir also




In [15]:
from llvmlite import ir
import llvmlite.binding as llvm
int_type    = ir.IntType(32)

#def of 


def to_llvm(f : smt.FuncDeclRef):
    ir.Function(module, func_type, name)


In [ ]:
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()  # yes, even this one
target = llvm.Target.from_default_triple()
target_machine = target.create_target_machine()
backing_mod = llvm.parse_assembly("")
engine = llvm.create_mcjit_compiler(backing_mod, target_machine)

mod = ir.Module()
mod = llvm.parse_assembly(str(module))
mod.verify()

pmb = llvm.PassManagerBuilder()
pmb.opt_level=3
pmb.loop_vectorize = True

pm = llvm.ModulePassManager()

NameError: name 'module' is not defined

In [ ]:
def to_lltype(sort : smt.SortRef):
    if sort == smt.BoolSort():
        return ir.IntType(1)
    elif isinstance(sort, smt.BitVecSortRef):
        return ir.IntType(sort.size())
    

def compile(defn):
    fn_ty = ir.FunctionType(to_lltype(defn.body.sort()), [to_lltype(arg.sort()) for arg in args])
    fn = ir.Function(mod, fn_ty, name=defn.name)
    env = {arg : fn.args[n] for n, arg in enumerate(defn.args)}
    block = fn.append_basic_block("entry")
    b = ir.IRBuilder(block)
    res = compile_expr(b, env, defn.body)
    b.ret(res)


def of_decl(f : smt.FuncDeclRef) -> :
    fn_ty = ir.FunctionType(to_lltype(f.range()), [to_lltype(arg) for arg in f.domain()])


def compile_expr(builder, env, e : smt.ExprRef):
    if e in env:
        return env[e]
    else:
        if smt.is_app(e):

            builder.call(of_decl(e.decl()) , tail="musttail")
        if smt.is_add(e):
        elif smt.is_ite(e):
            builder.select(compile_expr(builder, env, e.arg(0)),
                            compile_expr(builder, env, e.arg(1)),
                            compile_expr(builder, env, e.arg(2)))
        elif smt.is_value(e):
            if smt.is_bool(e):
                return ir.Constant(ir.IntType(1), 1 if e.as_bool() else 0)
            elif smt.is_bv(e):
                return ir.Constant(ir.IntType(e.size()), e.as_long())
            else:
                raise NotImplementedError(f"no constant for {e}")





    

In [27]:
from llvmlite import ir, binding
import ctypes, sys

# 1) Expose CPython symbols to the JIT
pythonapi = ctypes.pythonapi
PyLong_FromLong = pythonapi.PyLong_FromLong
PyLong_FromLong.restype = ctypes.py_object  # convenient boxing for demo
PyLong_FromLong.argtypes = [ctypes.c_long]

#binding.initialize()
binding.initialize_native_target()
binding.initialize_native_asmprinter()
binding.add_symbol("PyLong_FromLong", ctypes.cast(PyLong_FromLong, ctypes.c_void_p).value)

# 2) Build a tiny module that calls PyLong_FromLong(long) -> PyObject*
mod = ir.Module()
i64 = ir.IntType(64)
voidptr = ir.IntType(8).as_pointer()  # stand-in for PyObject*

fn_ty = ir.FunctionType(voidptr, [i64])
fn = ir.Function(mod, fn_ty, name="make_int")

callee_ty = ir.FunctionType(voidptr, [i64])
callee = ir.Function(mod, callee_ty, name="PyLong_FromLong")

block = fn.append_basic_block("entry")
b = ir.IRBuilder(block)
res = b.add(fn.args[0], ir.Constant(i64, 1))
res = b.call(callee, [res])

b.ret(res)

llvm_ir = str(mod)

# 3) JIT and run
target = binding.Target.from_default_triple()
tm = target.create_target_machine()
backing_mod = binding.parse_assembly(llvm_ir)
backing_mod.verify()
engine = binding.create_mcjit_compiler(backing_mod, tm)
engine.finalize_object()

addr = engine.get_function_address("make_int")
MakeInt = ctypes.CFUNCTYPE(ctypes.py_object, ctypes.c_long)(addr)

# GIL is held in Python here, so we can return a real PyObject safely:
obj = MakeInt(123)
print(obj, type(obj))  # -> 123 <class 'int'>


124 <class 'int'>


# Instability

Proofsmith - Michigan. Can mutate proofs to figure out what triggers you should have, try to simplify proofs. Try to be self improving. Dafny


relevancy propagation is an improtant parameter?
CVC5 sumit
smt vs sat solver.
Tactic choices

I should collect my instability exmaples

Triggers. I haven't been using them. Everything else smt based is all about them?

simp.add()  should make lhs a trigger also.

Tpot
SMT25 a conjecture

Can Cebeci disganosiung instability




In [ ]:
def trigger(pf : kd.Proof, trigger : smt.Pattern) -> kd.Proof:
    assert isinstance(pf, kd.Proof)
    thm = pf.thm
    assert isinstance(thm, smt.QuantifierRef) and thm.is_forall()
    # something.

# replace trigger? Have defn trigger by default on lhs.



# contextvars
https://docs.python.org/3/library/contextvars.html

Register a file local context, or explciitly hand it to Lemma or Goal


ctx = kd.Ctx()

x = ctx.add_freshvar(smt.FreshVar())
x = ctx.FreshVar()
  = ctx.FreshVars()


ctx.Goal()

ctx.Lemma()

Locale()


class Lemma
    bigctx



What a weird api.
You can use this to do implicit returns also


In [31]:
import contextvars

ctx = contextvars.Context()
v1 = contextvars.ContextVar("foo")
list(ctx.keys())

v1.set(10)
print(v1.get())
ctx = contextvars.copy_context()

@ctx.run
def _():
    print(list(contextvars.copy_context().items()))
    print(v1.get())
    v1.set(42)
    print(contextvars.copy_context()[v1])
    print(ctx.get(v1))
    assert v1.get() == 42

print("get", v1.get())
list(ctx.keys())
{
    "foo": v1
}

10
[(<ContextVar name='foo' at 0x7d96463c0b80>, 10), (<ContextVar name='foo' at 0x7d96464e2980>, 10), (<ContextVar name='foo' at 0x7d964662a9d0>, 10), (<ContextVar name='foo' at 0x7d9646649350>, 10), (<ContextVar name='foo' at 0x7d964651d350>, 10), (<ContextVar name='foo' at 0x7d96463c6750>, 10), (<ContextVar name='foo' at 0x7d96463c0d60>, 10), (<ContextVar name='foo' at 0x7d9646381a80>, 10), (<ContextVar name='foo' at 0x7d964646b4c0>, 10), (<ContextVar name='foo' at 0x7d96463c04f0>, 10), (<ContextVar name='parent_header' at 0x7d9668c97e70>, {'date': datetime.datetime(2025, 10, 2, 22, 11, 19, 20000, tzinfo=tzutc()), 'msg_id': '42165bb4-4357-4931-8f6e-aec77afd02a0', 'msg_type': 'execute_request', 'session': '2e88a59e-ef1e-402b-bc45-caa34516e0f0', 'username': '194a7128-5aa1-46bf-b35f-239ed23844a2', 'version': '5.2'}), (<ContextVar name='foo' at 0x7d96464e3a10>, 10), (<ContextVar name='foo' at 0x7d96463c4220>, 10), (<ContextVar name='foo' at 0x7d9668c94c20>, 10), (<ContextVar name='foo' a

{'foo': <ContextVar name='foo' at 0x7d964662a9d0>}

# Locales

So locales are like named contexts we can be in?
That is different from named collections of truths, but related.
Locales aren't proofs. They're like a second big context to be in?

ctx = Locale()
ctx

https://proofassistants.stackexchange.com/questions/23/what-are-locales-in-isabelle
https://lawrencecpaulson.github.io/2022/03/23/Locales.html locales are predicates?

group()




In [ ]:
@dataclass
class Context():
    fixes : list[smt.ExpRef]
    assumes : list[smt.ExpRef]

    def obtains(self, n): #?

def obtains(vs, fm):
    


# unsound


If I switched to tracking an id set,
why even have FreshVar judgement anymore?
Why have a Proof judgement? I could just track ids of proven theorems?


Is the herb form also unsound?




In [7]:
from kdrag.all import *
# Bascially I can't have the ability to skolem and universal var because they enable me to swap exists and forall.
N = kd.FreshVar("N", smt.IntSort())
x = smt.Const("x", smt.IntSort())
pf1 = kd.prove(smt.Exists([x], x > N))
xs, pf2 = kd.tactics.skolem(pf1)
pf3 = pf2.forall([N])(xs[0] + 1)
pf3
kd.prove(smt.BoolVal(False), by=[pf3])

LemmaError: (False, 'Countermodel', [N!23 = 3, f!26 = [1 -> 2, else -> 0]])

In [1]:
frozenset()

frozenset()

In [5]:
EMPTY = frozenset()
def build():
    return frozenset()
def refer():
    return EMPTY

Definitions need to be ALLVAR, but having a var in the definition is a big nono.


In [ ]:
%%timeit

x = frozenset()

27.3 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [4]:
%%timeit
x = EMPTY

7.32 ns ± 0.0199 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [6]:
%%timeit
build()

53.5 ns ± 0.903 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [7]:
%%timeit
refer()

18.7 ns ± 0.112 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [8]:
%%timeit
EMPTY | EMPTY

31.8 ns ± 0.204 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [9]:
%%timeit
EMPTY if len(EMPTY) == 0 else EMPTY | EMPTY

17.7 ns ± 0.0467 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


A formula has an implicit opening quantifier chain
exists constants, forall freshvars

A formula can always be weakened to 
exists constants, forall fvs <= freshvars

The implicit prefix grows with time (?) Or rather, because the freshvariables don't exist yet, also any formula can be strengthened to placing forall futurevars anyhwere
In particular
exists constants, forall freshvars, forall futurevars

Everything is considered auto strengthened to the currentdepth.
Every formula stands for a set of formulas between the qdepth and currentdepth of quantifiers.

{ exists cs, exists futurecs, forall freshvars,  forall futurevs, forall futurevars in possiblevars}

An existenail proof to be skolemized needs to have a combination of weakening and pulling through applied
exists cs, forall freshvars, exists vs =weaken> exists cs, forall fvs, exists vs  =pull>  exists cs, vs(fvs) forall fvs, 

exists cs, forall freshvars, exists vs =future_strengthen> exists cs, forall freshvars, forall futurevars, exists vs =weaken> 


Every Proof is actually an abstract domain of non implicit quantifier proofs.
This is because I don't want to have to reprove or carryu massive sets of all fresh variables around.


Making the weakening choice at the skolem point is nicer than making it at the

Definitions are kind of skolemizations of   exists t, t = body, fixes=args  and then apply generalization to args.
So definition axioms should get the currentdepth. There really shouldn't be variables in their body.

unfold 

https://arxiv.org/abs/2010.07516 julia world age
macro phase level
"isn't what you're doing macro programming" yes...
"global method table" What if I did dispatch through defns?
death age. could make overwriting defns sound? That'd be crazy.
a shadowing event in the implicit prefix
exists f1, exists f1

generalize is merely actualizing the implicit foralls as actual foralls.

Other abstract domains that might be nice for variable sets? Intervals, cofinite, top, depth=0 is top.

Graham seemed to immeidately want to use the term "Free Variable" https://en.wikipedia.org/wiki/Free_variables_and_bound_variables . To me free variable is an open lambda body which is rigid, not instantiable. FreshConst / FreshVar distinction makes sense to me. 

In [3]:
max([], default=0)

0

In [6]:
frozenset.intersection(frozenset([1,2]), frozenset([2,3]), frozenset([2,3]))

frozenset({2})

It isn't seaprable
The set of free vars is   `explicit_vars U [age, ...]`



In [ ]:
def merge(x,y):
    agex, fvx = x
    agey, fvy = y
    if agex > agey:
        extra = [v for v in fvy if v.age > agex]
    elif agex < agey:
        extra = [v for v in fvx if v.age > agey]
    max(agex,agey), (fvx & fvy) | extra
    

In [ ]:
def merge(ages):
    newage = 0
    fresh_vars = set()
    for age, fvs in ages:
        newage = max(age, newage)


In [ ]:



type QuantDepth = int
EMPTYSET : frozenset[smt.ExprRef] = frozenset()
class Proof():
    genvars : tuple[QuantDepth, frozenset[smt.ExprRef]] # quantified depth + exceptions. forall excepts, forall xfuture + ...
    # age
    # fresh_vars : frozenset[smt.ExprRef] = EMPTYSET

def merge(x, y):
    qdx, exceptsx = x.genvars
    qdy, exceptsy = y.genvars
    return max(qdx, qdy), exceptsx | exceptsy


def prove(thm, bys):
    qdepth = functools.reduce(merge, (by.genvars[0] for by in bys), (currentdepth, frozenset()))
    Prove(tuple[qdepth, ])

def axiom(thm):
    return Proof((currentdepth, frozenset()))

def generalize(pf, vs):
    qdepth, excepts = pf.genvars
    assert all(v.era >= pf.era or v in pf.genvars[1] for v in vs) # only generalize allowable variables
    return axiom(smt.ForAll(vs, pf.thm), pf.era, pf.fixes) # we do not necessarily lose power here.

def weaken(pf, qdepth=None, fixes=None):
    if qdepth is None:
        qdepth = currentdepth
    else:
        assert qdepth >= pf.genvars[0]
    if fixes is None:
        fixes = frozenset()
    else:
        assert isinstance(fixes, frozenset)
        assert all(fix.qdepth >=  pf.genvars[0] or fix in pf.genvars[1] for fix in fixes)
    return Proof((qdepth, frozenset(fixes)))


def skolem(pf, fixes):
    # skolem loses the ability to generalize. Only things _explicitly_ excepted are generalizable
    # currentdepth is the worst you can ever do.
    FreshFunction
    axiom(      , (currentdepth, frozenset(fixes) | pf.genvars[1]))

def FreshVar(name, sort):
    currentdepth += 1
    FreshEvidence(currentdepth, v)
    return v

In [ ]:

class AllVars(): ...
_ALLVARS = AllVars()

type GenVars = AllVars | frozenset[smt.ExprRef]

class Proof():
    genvars : GenVars # fixes?

def min_genvars(x : GenVars, y : GenVars) -> GenVars:
    if x is _ALLVARS:
        return y
    if y is _ALLVARS:
        return x
    assert isinstance(x, set) and isinstance(y, set)
    return x & y


def prove(fm : smt.BoolRef, by : list[Proof] = [], fixes=_ALLVARS) -> Proof:
    genvars = functools.reduce(min_genvars, (pf.genvars for pf in by), fixes)


def axiom(fm : smt.BoolRef, fixes : GenVars = frozenset()) -> Proof:
    assert isinstance(fm, smt.BoolRef)
    assert genvars is _ALLVARS or (isinstance(genvars, frozenset) and all(is_fresh_var(v) for v in genvars)) 
    return Proof( genvars, smt.Theorem(fm),)

def generalize(vs, pf):
    assert pf.fixes is _ALLVARS or all(v in pf.fixes for v in vs)
    axiom(smt.ForAll(list(vs), pf.thm.body), fixes=pf.fixes)

def weaken(pf, vs):
    assert isinstance(pf, Proof)
    assert pf.fixes is _ALLVARS or all(v in vs for v in pf.fixes)
    return Proof(vs, pf.thm)

def skolem(pf, fixes=frozenset()):
    assert isinstance(pf, Proof)
    assert isinstance(fixes, frozenset) and all(is_fresh_var(v) for v in fixes)
    assert pf.fixes is _ALLVARS or all(v in fixes for v in pf.fixes)
    if len(fixes) == 0:
        skolems =  [
        smt.FreshConst(q.var_sort(i), prefix=q.var_name(i).split("!")[0])
        for i in range(q.num_vars())
        ]
    else:
        genvs = list(fixes)
        sorts = [v.sort() for v in genvs]
        skolems = [smt.FreshFunction(*sorts, q.var_sort(i))(*genvs) for i in range(q.num_vars())]
    








In [ ]:
import logging

logger = logging.getLogger("knuckledragger")
"""
Proof_new = Proof.__new__

def sin_check(cls, thm, reason, admit=False, i_am_a_sinner=False):
    if admit and not config.admit_enabled:
        raise ValueError(
            thm, "was called with admit=True but config.admit_enabled=False"
        )
    if not i_am_a_sinner:
        raise ValueError("Proof is private. Use `kd.prove` or `kd.axiom`")
    return Proof_new(cls, thm, list(reason), admit)


Proof.__new__ = sin_check
"""
    # TODO: This is getting too hairy for the kernel? Reassess. Maybe just a lambda flag? Autolift?
    if lift_lambda and isinstance(body, smt.QuantifierRef) and body.is_lambda():
        # It is worth it to avoid having lambdas in definition.
        vs = fresh_const(body)
        # print(vs, f(*args)[tuple(vs)])
        # print(smt.substitute_vars(body.body(), *vs))
        def_ax = axiom(
            smt.ForAll(
                args + vs,
                smt.Eq(
                    f(*args)[tuple(vs)], smt.substitute_vars(body.body(), *reversed(vs))
                ),
            ),
            by="definition",
        )
    # assert f not in __sig or __sig[f].eq(   def_ax.thm)  # Check for redefinitions. This is kind of painful. Hmm.
    # Soft warning is more pleasant.

def test_lambda_def():
    # test that the lambda has been removed by the definition mechanism
    x, y = smt.Ints("x y")
    z, w = smt.Bools("z w")
    test = kd.define("test", [x], smt.Lambda([x], x), lift_lambda=True)
    assert test.defn.thm.body().eq(smt.ForAll([x, y], test(x)[y] == y).body())


def forget(ts: Iterable[smt.ExprRef], pf: Proof) -> Proof:
    """
    "Forget" a term using existentials. This is existential introduction.
    This could be derived from forget2
    """
    # Hmm. I seem to have rarely been using this
    assert is_proof(pf)
    vs = [smt.FreshConst(t.sort()) for t in ts]
    return axiom(smt.Exists(vs, smt.substitute(pf.thm, *zip(ts, vs))), ["forget", pf])

# modules

https://types.pl/@maxsnew/115305568271533541
a module of a sginature os a model of the signature
STLC[Sigma] parametrized lambda calculus
functors are functors

https://mathstodon.xyz/@olynch/115299725196735176

https://dl.acm.org/doi/pdf/10.1145/800055.802036

https://dl.acm.org/doi/pdf/10.1145/512644.512670  Using Dependent Types to Express Modu|ar Struetr~re

https://link.springer.com/chapter/10.1007/3-540-55719-9_102 A proposed categorical semantics for Pure ML

Currently knuckeldragger is C-like in that everything can clash. discipline can 
resgiter a name as _taken_ at the least.
kd.DeclareFun()

what if defns could be hierarchical? Theorems only apply in the right module? Seems kind of easy to puncture.
Maybe this could also be used to organize simps and auto.

I had notes on modules for egglog


First class modules. If I could internalize a notion of genericity or sorts. The sexp model.

https://pvs.csl.sri.com/doc/interpretations.pdf Interpretations in pvs

"parametric modules" Is that that mnuch different


# llvmlite compile
https://github.com/sdiehl/numpile/blob/master/numpile.py



# sexp

Another idea - 
delaboration via sexp

Also strangely enough, maybe syntactic manipulation might be faster via serialize -> sexp -> deserialize.
Building z3py stuff is just so slow.

Building rewriters over sexp is kind of interesting anyway

The transfer principle?

https://types.pl/@sandmouth/114904803899189629 What I did here is a little different.
Hmm.


s-expressions have set of scope and line number info



In [ ]:
class Syntax(list):
    start: int
    end: int
    scope_set = 

s = Syntax([1,2,3])



AttributeError: 'Syntax' object has no attribute 'start'

In [ ]:
Atom = Sexp1.constructor(0)
List = Sexp1.constructor(1)
_ax = smt.Unit(Sexp1.Atom(smt.StringVal("x")))
Unitdecl = _ax.decl()
Concatdecl = smt.Concat(_ax, _ax).decl()
Empty = smt.Empty(smt.SeqSort(Sexp1))


def pprint(e: smt.ExprRef) -> str:
    """
    Pretty print an s-expression.

    >>> pprint(parse("(+ a (b c))"))

    """
    assert e.sort() == Sexp1
    if e.decl() == Atom:
        s = e.arg(0)
        assert smt.is_string_value(s)
        return s.as_string()
    elif e.decl() == List:
        seq = e.arg(0)
        decl = seq.decl()
        if e.eq(Empty):
            return "()"
        elif decl == Unitdecl:
            return "(" + pprint(seq.arg(0)) + ")"
        elif decl == Concatdecl:
            return "(" + " ".join(pprint(c.arg(0)) for c in seq.children()) + ")"
        else:
            raise TypeError(f"Unexpected sequence {seq}")
    else:
        raise TypeError(f"Unexpected expression {e}, {e.decl().name()}")

# nominal
How to do nominal knuckeldragger? Semantic model?

Could help me understand slotted

https://www.cl.cam.ac.uk/~amp12/papers/nomlfo/nomlfo-draft.pdf

https://www.cl.cam.ac.uk/~amp12/talks/tacs01.pdf talk

No, maybe Tuple(Prim, AtomSet)


In [ ]:
# oof this is unsopund in the presence on FreshVars. Huh. Subtle or obvious?
def FreshAtom(xs, prefix=None):
    """
    Axiom schema for there are always new constants.
    >>> FreshAtom([a, b], prefix="fred")
    (fred!..., |= Distinct(fred!..., a, b))
    """
    # TODO: might also need a Seq form of
    z = smt.FreshConst(Atom, prefix=prefix)
    return z, kd.axiom(smt.Distinct(z, *xs))

# But this is the same thing. Uh oh.
def DistinctN(n : int):
    xs = [smt.Const(f"x{i}", Atom) for i in range(n)]
    z = smt.Const("z", Atom)
    return kd.axiom(smt.ForAll([xs], smt.Exists([z], Distinct(z, *xs))))

# Is FreshVar unsound?
# I can prove something exists
# forall exists does not imply exists forall








|= False

In [ ]:
Atom = smt.DeclareSort("Atom")


x,y,z = smt.Consts("x y z", Atom)
kd.define("swap", [x,y,z], 
    smt.If(x == z, y,
    smt.If(y == z, x,
    z))
)

swap = kd.notation.SortDispatch(name="swap")
free = kd.notation.SortDispatch(name="free")

swap.register()

def NomSort(s : smt.SortRef) -> smt.SortRef:
    dt = kd.Inductive("Nom_" + s.name(), [s])
    dt.declare("Atom", ("atom", Atom)) # Set(Atom) ? Var?
    dt.declare("Value", ("prim", s))
    dt = dt.create()
    v = smt.Const("v", dt)
    swap.define([s,x,y], 
        smt.If(dt.is_Atom(v), dt.Atom(swap(x,y,dt.prim(v))),
        dt.Value(v))
    )
    # v # t
    distinct.define([s,x],
        smt.If(dt.is_Atom(v), v != dt.atom(x),
        smt.BoolVal(False)
    ))

    return dt.create(), swap, free



#  ??????.  phi(x,y,z) ?    (Bool, Set(Atom)) ? Perm -> Bool ?  
Bool = NormSort(smt.BoolSort())

f = smt.ArrayConst("f", Atom, Atom)
is_iso = kd.define("is_iso", [f], 
    smt.ForAll([x],
        smt.Exists([y],
            f[x] == y
        )
    )
)
compact = kd.define("compact", [f],
    smt.Exists([seq],
        f[a] == smt.In(seq, a)
    )
    smt.ForAll([x,y,z],
        smt.Implies(f[x] == f[y], x == y)
    )
)

perm_inv = kd.define("perm_inv", [a],
    smt.ForAll([x,y],
        smt.Implies(free(a,x) == free(a,y), x == y)
    )
)




In [ ]:
NomVar = smt.DeclareSort("NomVar")

def Nominal(S : smt.SortRef):
    dt = kd.Inductive("Nom_" + S.name())
    dt.declare("Nom", ("var", NomVar))
    dt.declare("Const", ("val", S))
    return dt.create()

# not good enoguh for recursive trees.

def NomDT(name):
    dt = kd.Inductive("name")
    dt.declare("Var", ("var", NomVar))
    return dt

# but also it has a different 

# Conv

Conv could be nice way to use zippers + amgnual congruence rules.
Which in turn let's us maybe get a handle on generalized congruence.

Lenses?


https://pavpanchekha.com/blog/zippers/multi-zippers.html
https://stackoverflow.com/questions/3436559/zipper-like-data-structure-with-more-than-one-cursor
https://okmij.org/ftp/continuations/zipper.html#zipper2 zippers with several holes



# Knuckle Kernels
a monorepo of knuckeldragger kernels



# Delab

lisp2tex?


pprint?






In [ ]:

pprint_binop = {
    "and": r"\land",
    "or": r"\lor",
    "not": r"\lnot",
    "=>": r"\implies",
    "=": "=",
    "distinct": r"\neq",
    "+": "+",
    "-": "-",
    "*": r"\cdot",
    "/": "/",
    "<": "<",
    "<=": r"\leq",
    ">": ">",
    ">=": r"\geq",
}
pprint_unop = {
    "not": r"\lnot",
    "-": "-",
}



#def pprint(sexp):
def latex(sexp):
    match sexp:
        case [f, *args]:



# macro smt


# parametrized solvers

grind can take in theories


In [ ]:
from typing import Protocol
class Theory(Protocol):
    decls : list[smt.FuncDeclRef]
    def init(self) -> list[smt.BoolRef]: ...
    def preprocess(self, formula : smt.BoolRef):
        # callback from Solver.add

class Ring(Protocol):
    ...

class RingTheory(Theory):
    ring : Ring

    def preprocess(self, formula): ...
        # do ring normalization
    def simplify(self, e : smt.ExprRef): ...

class Ring(Protocol, Theory):



class Solver():
    solver : smt.Solver
    theories : list[Theory]
    def add_theory(self, t : Theory):
        self.theories.add(t)
        for e in t.init():
            self.solver.add(e)
    def add(self, formula):
        self.solver.add(t)
        for theory in theories:
            for t in theory.preprocess(formula):
                self.solver.add(t)
    def check(self):
        res = self.solver.check()
        if res == smt.unsat:
            return smt.unsat
        elif res == smt.sat:
            m = self.solver.get_model()
            for theory in self.theories:
                theory.check(m)




# delta satisfiability

https://scungao.github.io/papers/delta_complete_decision_procedures.pdf

nonstyandard reals
lifting

nate says maxmimize delta is nice.

In [ ]:

def unfold_old(e: smt.ExprRef, decls=None, trace=None) -> smt.ExprRef:
    """
    Do a single unfold sweep, unfolding definitions defined by `kd.define`.
    The optional trace parameter will record proof along the way.
    `decls` is an optional list of declarations to unfold. If None, all definitions are unfolded.

    >>> x = smt.Int("x")
    >>> f = kd.define("f", [x], x + 42)
    >>> trace = []
    >>> unfold_old(f(1), trace=trace)
    1 + 42
    >>> trace
    [|= f(1) == 1 + 42]

    >>> unfold_old(smt.Lambda([x], f(x)))
    Lambda(x, x + 42)
    """
    if smt.is_app(e):
        decl = e.decl()
        children = [unfold(c, decls=decls, trace=trace) for c in e.children()]
        defn = kd.kernel.defns.get(decl)
        if defn is not None and (decls is None or decl in decls):
            e1 = smt.substitute(defn.body, *zip(defn.args, children))
            e = e1
            if trace is not None:
                if isinstance(defn.ax.thm, smt.QuantifierRef):
                    trace.append((defn.ax(*children)))
                # else:
                # TODO: It would be better to only emit the forall axiom when necessary.
                # When we have traversed under a binder, things get complicated.
                # instantiating the axiom on the fresh variable will not be useful.
                # so just emitting the needed axiom is a hack for now.
                # trace.append(defn.ax)
            return e1
        else:
            return decl(*children)
    elif isinstance(e, smt.QuantifierRef):
        vs0 = [smt.Const(e.var_name(i), e.var_sort(i)) for i in range(e.num_vars())]
        vs, e1 = kd.utils.open_binder(e)
        if trace is not None:
            subtrace = []
            e2 = unfold(e1, decls=decls, trace=subtrace)
            if not e2.eq(e1):
                trace.extend([pf.forall(vs) for pf in subtrace])
        else:
            e2 = unfold(e1, decls=decls, trace=None)
        if e.is_forall():
            # TODO: When we go under a quantifier, any trace breadcrumb should really be re-quantified in order to be useful.
            # maybe herbrandize?
            e3 = smt.ForAll(vs, e2)
        elif e.is_exists():
            e3 = smt.Exists(vs, e2)
        elif e.is_lambda():
            e3 = smt.Lambda(vs, e2)
        else:
            raise Exception("Unexpected quantifier", e)
        t, pf = kd.kernel.rename_vars(e3, vs0)
        if trace is not None and not t.eq(e):
            trace.append(pf)
        return t
    else:
        return e

# analysis

rewrite should be improved to go under binders.
The rewrite tactic doesn't use rewrite facilities?



In [8]:
from kdrag.all import *
import kdrag.theories.real as real
from kdrag.theories.real.seq import *
@kd.Theorem(smt.Implies(has_lim(a, x) & has_lim(b, y), has_lim(a + b, x + y)))
def has_lim_add(l):
    l.unfold(has_lim)
    l.intros()
    l.split(0)
    eps = l.fix()
    l.intros()
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(a[n] - x) < eps / 2)))
    l.apply(0)
    l.auto()
    N1 = l.obtain(-1)
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(b[n] - y) < eps / 2)))
    l.apply(1)
    l.auto()
    N2 = l.obtain(-1)
    l.exists(smt.If(N1 > N2, N1, N2))
    n1 = l.fix()
    l.instan(0, n1)
    l.instan(1, n1)
    l.unfold(add)
    l.auto(by=[real.abs.defn])


@kd.PTheorem(smt.Implies(has_lim(a, x) & has_lim(b, y), has_lim(a - b, x - y)))
def has_lim_sub(l):
    l.unfold(has_lim)
    l.intros()
    l.split(0)
    eps = l.fix()
    l.intros()
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(a[n] - x) < eps / 2)))
    l.apply(0)
    l.auto()
    N1 = l.obtain(-1)
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(b[n] - y) < eps / 2)))
    l.apply(1)
    l.auto()
    N2 = l.obtain(-1)
    l.exists(smt.If(N1 > N2, N1, N2))
    n1 = l.fix()
    l.instan(0, n1)
    l.instan(1, n1)
    l.unfold(sub)
    l.auto(by=[real.abs.defn])

Lemma Complete! Change PTheorem to Theorem


In [1]:
@kd.PTheorem(smt.Implies(has_lim(a,x) & has_lim(b, y), has_lim(a + b, x + y)))
def has_lim_add(l):
    l.unfold(has_lim)
    l.intros()
    l.split(0)
    eps = l.fix()
    l.intros()
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(a[n] - x) < eps / 2)))
    l.apply(0)
    l.auto()
    N1 = l.obtain(-1)
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(b[n] - y) < eps / 2)))
    l.apply(1)
    l.auto()
    N2 = l.obtain(-1)
    l.exists(smt.If(N1 > N2, N1, N2))
    n1 = l.fix()
    l.instan(0,n1)
    l.instan(1,n1)
    l.unfold(add)
    l.auto(by=[real.abs.defn])





NameError: name 'kd' is not defined

In [ ]:
def lift_lim(lim, N, by=[]):


Once I have more than 3 facts in context, giving them labels would be nice in printing.


In [4]:

from kdrag.all import *
import kdrag.theories.real as real
from kdrag.theories.real.seq import *

@kd.PTheorem(smt.Implies(has_lim(a,x) & has_lim(b, y), has_lim(a - b, x - y)))
def has_lim_sub(l):
    l.unfold(has_lim)
    l.intros()
    l.split(0)
    eps = l.fix()
    l.intros()
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(a[n] - x) < eps / 2)))
    l.apply(0)
    l.auto()
    N1 = l.obtain(-1)
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(b[n] - y) < eps / 2)))
    l.apply(1)
    l.auto()
    N2 = l.obtain(-1)
    l.exists(smt.If(N1 > N2, N1, N2))
    n1 = l.fix()
    l.instan(0,n1)
    l.instan(1,n1)
    l.unfold(sub)
    l.auto(by=[real.abs.defn])

Lemma Complete! Change PTheorem to Theorem


# C parsing

In [15]:
from pycparser import parse_file, c_generator
import pycparser

text = """
int myadd(int x, int y) {
    return x + y;
}
"""

parser = pycparser.c_parser.CParser()
ast = parser.parse(text)
ast.show()
generator = c_generator.CGenerator()
generator.visit(ast)

FileAST: 
  FuncDef: 
    Decl: myadd, [], [], [], []
      FuncDecl: 
        ParamList: 
          Decl: x, [], [], [], []
            TypeDecl: x, [], None
              IdentifierType: ['int']
          Decl: y, [], [], [], []
            TypeDecl: y, [], None
              IdentifierType: ['int']
        TypeDecl: myadd, [], None
          IdentifierType: ['int']
    Compound: 
      Return: 
        BinaryOp: +
          ID: x
          ID: y


'int myadd(int x, int y)\n{\n  return x + y;\n}\n\n'

# rewrite
forall cong

going into branches of if, we collect local cond.
Wait but actually a lot of this is hypothetical.
and if we need to 

def hyp_cong(f):
    ks.axiom(t1 == t2 => f(t1) == f(t1))

The analog of herb

def lambda_cong():
    vs, kd.axiom(body1 == body2 -> lambda vs, body = lambda vs, body2)
def subst():
    lhs == rhs -> t1 == t2


tacticals
repeat
branching subgoals. apply rw at each

make subproofs and them meld them?





In [ ]:
def rewrite1_rec(t, vs, lhs, rhs) -> Optional[kd.Proof]:
    """

    >>> x,y = smt.Ints("x y")
    >>> pf = kd.prove(smt.ForAll([x], x + 0 == x))
    >>> rewrite1_rec(x + 0 + 0, [x], x + 0, x)
    """

    def worker(t):
        subst = utils.pmatch(vs, lhs, t)
        if subst is not None:
            return kd.kernel.instan(pf, [subst[v] for v in vs])
        elif isinstance(t, smt.QuantifierRef):
            vs1, body = kd.utils.open_binder_unhygienic(t)
            pf = worker(body)
            if pf is None:
                return None
            else:
                pf = kd.kernel.subst(t, pf)
                if t.is_forall():
                    return kd.kernel.forall_cong(vs1, pf)
                elif t.is_exists():
                    return kd.kernel.exists_cong(vs1, pf)
                elif t.is_lambda():
                    return kd.kernel.lambda_cong(vs1, pf)
                else:
                    raise Exception("Unexpected quantifier", t)
        elif smt.is_app(t):
            for c in t.children():
                pf = worker(c)
                if pf is not None:
                    return pf
            return None
        else:
            raise Exception("Unexpected term in rewrite1", t)

    pf = worker(t)
    if pf is not None:
        return kd.kernel.subst(t, pf)
    else:
        return None

In [ ]:
def rewrite1_rec(t, pf : kd.Proof) -> Optional[kd.Proof]:
    """

    >>> x,y = smt.Ints("x y")
    >>> pf = kd.prove(smt.ForAll([x], x + 0 == x))
    >>> rewrite1_rec(x + 0 + 0, [x], x + 0, x)
    """

    if isinstance(pf, kd)
    vs, body = kd.utils.open_binder_unhygienic(pf.thm)
    assert smt.is_eq(body)
    lhs = body.arg(0)

    def worker(t):
        subst = utils.pmatch(vs, lhs, t)
        if subst is not None:
            return kd.kernel.instan(pf, [subst[v] for v in vs])
        elif isinstance(t, smt.QuantifierRef):
            vs1, body = kd.utils.open_binder_unhygienic(t)
            pf = worker(body)
            if pf is None:
                return None
            else:
                pf = kd.kernel.subst(t, pf)
                if t.is_forall():
                    return kd.kernel.forall_cong(vs1, pf)
                elif t.is_exists():
                    return kd.kernel.exists_cong(vs1, pf)
                elif t.is_lambda():
                    return kd.kernel.lambda_cong(vs1, pf)
                else:
                    raise Exception("Unexpected quantifier", t)
        elif smt.is_app(t):
            for c in t.children():
                pf = worker(c)
                if pf is not None:
                    return pf
            return None
        else:
            raise Exception("Unexpected term in rewrite1", t)

    pf = worker(t)
    if pf is not None:
        return kd.kernel.subst(t, pf)
    else:
        return None

In [ ]:

def rewrite1_rec(t, pf : kd.Proof) -> Optional[kd.Proof]:
    """

    >>> x,y = smt.Ints("x y")
    >>> pf = kd.prove(smt.ForAll([x], x + 0 == x))
    >>> rewrite1_rec(x + 0 + 0, [x], x + 0, x)
    """

    if isinstance(pf, kd)
    vs, body = kd.utils.open_binder_unhygienic(pf.thm)
    assert smt.is_eq(body)
    lhs = body.arg(0)

    def worker(t):
        subst = utils.pmatch(vs, lhs, t)
        if subst is not None:
            return kd.kernel.instan(pf, [subst[v] for v in vs])
        elif isinstance(t, smt.QuantifierRef):
            vs1, body = kd.utils.open_binder_unhygienic(t)
            pf = worker(body)
            if pf is None:
                return None
            else:
                pf = kd.kernel.subst(t, pf)
                if t.is_forall():
                    return kd.kernel.forall_cong(vs1, pf)
                elif t.is_exists():
                    return kd.kernel.exists_cong(vs1, pf)
                elif t.is_lambda():
                    return kd.kernel.lambda_cong(vs1, pf)
                else:
                    raise Exception("Unexpected quantifier", t)
        elif smt.is_app(t):
            for c in t.children():
                pf = worker(c)
                if pf is not None:
                    return pf
            return None
        else:
            raise Exception("Unexpected term in rewrite1", t)

    pf = worker(t)
    if pf is not None:
        return kd.kernel.subst(t, pf)
    else:
        return None


In [ ]:



def cond_rw(): ...

# Like have. Auto discharge via by. otherwise add to new goal to prove precondition
#




# CF Known Bit

# Category


In [ ]:
from typing import Protocol
from kdrag.all import *
class Category(Protocol):
    Ob : smt.SortRef
    Arr : smt.SortRef
    cod : smt.FuncDeclRef # cod : Arr -> Ob
    dom : smt.FuncDeclRef # dom : Arr -> Ob
    id_ : smt.FuncDeclRef # id : Ob -> Arr
    id_wf : kd.Proof # forall x, dom(id(x)) = x & cod(id(x)) = x
    comp : smt.FuncDeclRef # comp : Arr x Arr -> Arr
    comp_wf : kd.Proof # forall f g, dom(f) = cod(g) => dom(comp(f,g)) == dom(g) & cod(comp(f,g)) == cod(f)
    comp_id : kd.Proof # forall f, comp(f, id(dom(f))) = f 
    id_comp : kd.Proof # forall x, comp(id(cod(f)), f) = f
    comp_assoc : kd.Proof # forall f g h, comp(comp(f,g), h) = comp(f, comp(g,h)) 


#class ACategory(Category):
Ob = smt.DeclareSort("AOb")
Arr = smt.DeclareSort("AArr")
x,y,z = smt.Consts("x y z", Ob)
f,g,h,k = smt.Consts("f g h k", Arr)
cod = smt.Function("cod", Arr, Ob)
dom = smt.Function("dom", Arr, Ob)
id_ = smt.Function("id", Ob, Arr)
id_wf = kd.axiom(smt.ForAll([x], smt.And(dom(id_(x)) == x, cod(id_(x)) == x)))
comp = smt.Function("comp", Arr, Arr, Arr)
kd.notation.matmul.register(Arr, lambda f,g: comp(f,g))
comp_wf = kd.axiom(smt.ForAll([f,g], smt.Implies(dom(f) == cod(g), smt.And(dom(comp(f,g)) == dom(g), cod(comp(f,g)) == cod(f)))))
comp_id = kd.axiom(smt.ForAll([f], comp(f, id_(dom(f))) == f))
id_comp = kd.axiom(smt.ForAll([f], comp(id_(cod(f)), f) == f))
comp_assoc = kd.axiom(smt.ForAll([f,g,h], comp(comp(f,g), h) == comp(f, comp(g,h))))

def HasType(f, A, B):
    return smt.And(dom(f) == A, cod(f) == B)
has_type = kd.define("has_type", [f, x, y], HasType(f, x, y))
is_iso = kd.define("is_iso", [f], kd.QExists([g], dom(g) == cod(f), cod(g) == dom(f), f @ g == id_(dom(g))), g @ f == id_(cod(g)))

# Dual and Product Category
pair = kd.notation.SortDispatch(name="pair")
pair_ob = smt.Function("pair", Ob, Ob, Ob)
pair.register(Ob, pair_ob)
pair_arr = smt.Function("pair", Arr, Arr, Arr)
pair.register(Arr, pair_arr)

pair_wf = kd.axiom(kd.QForAll([f,g], dom(pair(f,g)) == pair(dom(f), dom(g)), cod(pair(f,g)) == pair(cod(f), cod(g))))
comp_pair = kd.axiom(kd.QForAll([f,g,h,k], dom(f) == cod(h), dom(g) == cod(k), pair(f,g) @ pair(h,k) == pair(f @ h, g @ k)))


ObSet = smt.ArraySort(Ob, smt.BoolSort())
obset = smt.Const("obset", ObSet)
ArrSet = smt.ArraySort(Arr, smt.BoolSort())
arrset = smt.Const("arrset", ArrSet)

Cat = kd.Struct("Cat", ("obset", ObSet), ("arrset", ArrSet), pred=
    lambda cat:smt.And(
    kd.QForAll([x,f], cat.arrset[f], smt.And(cat.obset[dom(f)], cat.obset[cod(f)])),
    kd.QForAll([x], cat.obset[x], cat.arrset[id_(x)]),
    kd.QForAll([f,g], cat.arrset[f], arrset[g], dom(f) == cod(g), cat.arrset[f @ g])
))
cat = smt.Const("cat", Cat)

is_cat = kd.define("is_cat", [cat],
smt.And(
    kd.QForAll([x,f], cat.arrset[f], smt.And(cat.obset[dom(f)], cat.obset[cod(f)])),
    kd.QForAll([x], cat.obset[x], cat.arrset[id_(x)]),
    kd.QForAll([f,g], cat.arrset[f], arrset[g], dom(f) == cod(g), cat.arrset[f @ g])
))







# Linear Space

# set zf

Maybe if I'd used elts more with built in set operations some stuff could be more automatic?

https://www.isa-afp.org/sessions/hol/#Fun.html pull other stuff from isabelle. Work backwards?

rework triggers

map_pos on rewrite.
rw could work under binders
rw in prove

https://isabelle.in.tum.de/dist/Isabelle2025/doc/logics-ZF.pdf

In [33]:
from kdrag.all import *
from kdrag.theories.logic.zf import *


@kd.PTheorem(smt.ForAll([x,a], (x <= sing(a)) == smt.Or(x == sing(a), x == emp)))
def le_sing(l):
    l.unfold(le)
    x,a = l.fixes()
    l.cases(x == emp)
    
    l.auto(by=[elem_sing, elem_emp])

    l.rw(-1).simp()
    l.split().intros()
    l.rw(ext_ax)
    x1 = l.fix()
    l.instan(-1, x1)
    l.rw(elem_sing, at=-1)
    l.rw(elem_sing)
    l.split()
    l.intros()
    l.auto()
    l.intros()
    l.rw(-1)
    #l.auto(by=[elem_sing, elem_emp])







Next Goal:
 [x!1743, a!1744, x!1748];
[(x!1743 == emp) == False,
 ForAll(x, Or(Not(elem(x, x!1743)), elem(x, sing(a!1744)))),
 Or(Not(elem(x!1748, x!1743)), x!1748 == a!1744),
 x!1748 == a!1744]
?|= elem(a!1744, x!1743)


In [ ]:
    #l.auto(by=[elem_sing, elem_emp])
    l.split()
    l.intros()
    #l.have(elem(a, sing(a)), by=[elem_sing])
    #l.have(smt.Not(elem(x, emp)), by=[elem_emp])
    #l.instan(0, a)
    l.cases(x == emp)
    l.case().auto(by=[elem_emp])
    l.rw(-1).simp()
    x1 = l.rw(ext_ax).fix()
    l.instan(0, x1)
    l.split()
    l.auto()
    l.rw(elem_sing)
    l.have(sing(a) != emp, by=[sing_not_emp])
    #l.auto()
    #l.auto(by=[elem_sing, elem_emp])

#kd.search(emp.decl())


In [7]:
@kd.PTheorem(smt.ForAll([a], pow(sing(a)) == sing(a) | emp))
def pow_sing(l):
    a = l.fix()
    l.rw(ext_ax).fix()
    l.rw(elem_pow).rw(elem_union).rw(elem_emp)

Next Goal:
 [a!399, x!402];
[] ?|= le(x!402, sing(a!399)) ==
Or(elem(x!402, sing(a!399)), False)


In [ ]:

    #l.auto(by=[inter_le])

#kd.search(le)

In [ ]:
from kdrag.all import *
from kdrag.theories.logic.zf import *


pow(upair(a,b)) == FinSet(emp, sing(a), sing(b), upair(a,b))
pow(sing(a)) == sing(a) | emp
pow(a & b) == pow(a) & pow(b)
pow(a | b) <= pow(a) | pow(b)


mul = kd.notation.mul.define([A,B], sep(pow(pow(A | B)), smt.Lambda([p], kd.QExists(elem(a,A), elem(b,B), p == pair(a,b)))))

elem_mul = kd.prove(smt.ForAll([A,B,a,b], elem(pair(a,b), A * B)) == smt.And(elem(a,A), elem(b,B)))
elem_mul = kd.prove(smt.ForAll([A,B,p], elem(p, A * B)) == smt.And(elem(fst(p), A), elem(snd(b),B))) # this one is more general.
elem_mul = kd.prove(smt.ForAll([A,B,p], elem(p, A * B)) == smt.Exists([a,b], elem(a,A), elem(b,B), p == pair(a,b)))
elem_mul_is_pair = kd.prove(kd.QForAll([A,B,p], elem(p, A * B), is_pair(p)))


(A | B) * X == (A * X) | (B * X)
X * (A | B) == (X * A) | (X * B)
(A & B) * X == (A * X) & (B * X)
X * (A & B) == (X * A) & (X * B)
(A - B) * X == (A * X) - (B * X)

A * emp == emp
emp * A == emp

smt.Implies(smt.And(A <= X, B <= Y), A * B <= X * Y)

dom = kd.define("dom", [R], sep(bigunion(bingunion(R)), smt.Lambda([a], kd.QExists([b], elem(pair(a,b)), R))))
ran = kd.define("ran", [R], sep(bigunion(bingunion(R)), smt.Lambda([b], kd.QExists([a], elem(pair(a,b)), R))))
#dom = kd.define("dom", [R], )
idrel = kd.define("idrel", [A], sep(A * A, smt.Lambda([a,b], a == b)))

has_type = kd.define("has_type", [R, A, B],  R <= A * B)
#is_refl = kd.define("is_refl", [R, X], has_type(R,X,X) & kd.QForAll([x], elem(x,X), elem(pair(x,x), R)))
is_refl = kd.define("is_refl", [R], kd.QForAll([x], smt.Exists([y], elem(pair(x,y), R) | elem(pair(y,x), R)), elem(pair(x,x), R)))
is_symm = kd.define("is_symm", [R], smt.ForAll([a,b], elem(pair(a,b), R), elem(pair(b,a), R)))
is_trans = kd.define("is_trans", [R], kd.QForAll([a,b,c], elem(pair(a,b), R) & elem(pair(b,c), R), elem(pair(a,c), R)))
is_equiv = kd.define("is_equiv", [R], smt.And(is_refl(R), is_symm(R), is_trans(R)))

is_equiv(idrel(A))
is_equiv(X * X)


is_partition = kd.define("is_partition", [P, A], smt.And(biginter(P) == emp, bigunion(P) == A))


is_fun = kd.define("is_fun", [R], kd.QForAll([a,b,c], elem(pair(a,b), R), elem(pair(a,c), R), b == c))
is_total = kd.define("is_total", [R, A], kd.QForAll([a], elem(a,A), smt.Exists([b], elem(pair(a,b), R))))

is_fun(idrel(A))

Family = smt.ArraySort(ZFSet, ZFSet)
# is internal
#is_intfun = kd.define("is_intfun", [F], smt.Exists([R], is_fun(R), smt.ForAll([x], F[x] == )

#is_set = kd.define("is_set", [P], smt.Exists([A], smt.ForAll([x], elem(x, A) == P[x])))





le(pow(union(a!0, b!1)), union(pow(a!0), pow(b!1)))

In [ ]:

@kd.Theorem(smt.Not(gencomp))
def russell(l):
    l.unfold(is_set)
    l.unfold(reflects)
    l.intros()
    l.instan(0, smt.Lambda([x], smt.Not(elem(x, x))))
    A = l.obtain(-1)
    l.instan(-1, A)
    l.auto()


In [ ]:
sing(a) | sing(b) == upair(a,b)
sing(a) | upair(a,b) == upair(a,b)

In [ ]:



    #l.unfold(pair)
    #l.rw(bigunion_upair)
    #l.have(upair(sing(a), upair(a,b)) != emp, by=[upair_not_emp])
    #l.rw(sing_union_upair)
    #l.have(smt.Or(pick(upair(a,b)) == a, pick(upair(a,b)) == b), by=[pick_upair])
    #l.cases(pick(upair(a,b)) == a)
    #l.have()
#kd.search(bigunion)


In [33]:
kd.search(union)

{('__main__.union_comm',
  |= ForAll([a!0, b!1], union(a!0, b!1) == union(b!1, a!0))): (),
 ('__main__.elem_union',
  |= ForAll([x, A, B],
         elem(x, union(A, B)) == Or(elem(x, A), elem(x, B)))): (),
 ('__main__.union_assoc',
  |= ForAll([a!0, b!1, c!2],
         union(union(a!0, b!1), c!2) ==
         union(a!0, union(b!1, c!2)))): (),
 ('__main__.union_idem', |= ForAll(a!0, union(a!0, a!0) == a!0)): (),
 ('__main__.le_union',
  |= ForAll([a!0, b!1, c!2], le(a!0, union(a!0, b!1)))): (),
 ('__main__.inter_union_distr',
  |= ForAll([a!0, b!1, c!2],
         inter(a!0, union(b!1, c!2)) ==
         union(inter(a!0, b!1), inter(a!0, c!2)))): (),
 ('__main__.union_inter_distr',
  |= ForAll([a!0, b!1, c!2],
         inter(union(a!0, b!1), c!2) ==
         union(inter(a!0, c!2), inter(b!1, c!2)))): (),
 ('__main__.bigunion_upair',
  |= ForAll([a!0, b!1],
         bigunion(upair(a!0, b!1)) == union(a!0, b!1))): (),
 ('kdrag.theories.logic.zf.union_comm',
  |= ForAll([a!0, b!1], union(a!0

In [ ]:

@kd.tactics.PTheorem(kd.QForAll([a], a != emp, kd.QExists([c], c != emp, elem(c, a)), a != sing(emp), bigunion(a) != emp))
def bigunion_not_emp(l):
    a = l.fix()
    l.intros()
    l.split(at=0)
    c = l.obtain(1)
    l.intros()
    l.rw(ext_ax, at=-1)
    l.have(elem(pick(a), a), by=[elem_pick])
    #l.have(elem(kd.QExists([d], elem(d, a)))
    #l.have(elem(pick(a), bigunion(a)), by=[bigunion_ax, elem_pick])
    

    
    #l.have(elem(pick(a), bigunion(a)))


TimeoutError: Timeout. Maybe you have given `prove` too many or not enough lemmas?

In [21]:
smt.is_distinct(bigunion(a) != emp)

True





#kd.search(bigunion)

In [16]:
kd.search(bigunion)



{('__main__.bigunion_ax',
  |= ForAll([A, x],
         elem(x, bigunion(A)) ==
         (Exists(B, And(elem(B, A), elem(x, B)))))): (),
 ('__main__.elem_bigunion_upair',
  |= ForAll([x, A, B],
         elem(x, bigunion(upair(A, B))) ==
         Or(elem(x, A), elem(x, B)))): (),
 ('kdrag.theories.logic.zf.bigunion_ax',
  |= ForAll([A, x],
         elem(x, bigunion(A)) ==
         (Exists(B, And(elem(B, A), elem(x, B)))))): (),
 ('kdrag.theories.logic.zf.elem_bigunion_upair',
  |= ForAll([x, A, B],
         elem(x, bigunion(upair(A, B))) ==
         Or(elem(x, A), elem(x, B)))): ()}

In [ ]:


    #l.auto(by=[pick_upair])
    #l.unfold   #l.rw(elem_sing)
    #l.auto()


In [ ]:

l = kd.Lemma(smt.ForAll([a,b,c,d], (pair(a,b) == pair(c,d)) == smt.And(a == c, b == d)))



kd.Lemma(fst(pair(a,b)) == a)

kd.Lemma(snd(pair(a,b)) == b)

@Theorem()

In [ ]:
ProofState = kd.tactics.ProofState

_TRUE = kd.kernel.prove(smt.BoolVal(True))


def Theorem(
    goal: smt.BoolRef,
) -> Callable[[Callable[[ProofState], None]], kd.kernel.Proof]:
    def res(f: Callable[[ProofState], None]) -> kd.Proof:
        l = kd.Lemma(goal)
        f(l)
        pf = l.qed()
        object.__setattr__(pf, "__doc__", getattr(f, "__doc__", None))
        object.__setattr__(pf, "__module__", getattr(f, "__module__", None))
        object.__setattr__(pf, "__name__", getattr(f, "__name__", None))
        object.__setattr__(pf, "__qualname__", getattr(f, "__qualname__", None))
    return res


x, y, z = smt.Ints("x y z")

@Theorem(x + 1 > x)
def inc_ge(l):
    "increment the stuff"
    l.auto()

def check(p: kd.Proof) -> kd.Proof:
    return p

assert inc_ge is not None
help(check(inc_ge))

Help on Proof in module __main__:

inc_ge = |= x + 1 > x
    incremenet the stuff



In [ ]:
@Lemma()
def elem_union(l):
    a,b,c,d = l.fix()
    l.auto()
    l.unfold(elem)
    l.rw(le)
    l.auto()
    


In [31]:
smt.ForAll([a,x], elem(x,a), patterns=[elem(x,a)])

ForAll([a!0, x], elem(x, a!0))

In [2]:
from kdrag.all import *
from kdrag.theories.logic.zf import *

#A, B, x, y, z = smt.Consts("A B x y z", zf.ZFSet)

#union_comm = #kd.prove(a | b == b | a, by=[bigunion_ax, upair_ax, ext_ax(a | b, b | a)])

l = kd.Lemma(a | b == b | a)
l.unfold(union)
l.rw(ext_ax)
_x = l.fix()
l.auto(by=[upair_ax, bigunion_ax])
union_comm = l.qed().forall([a,b])

l = kd.Lemma(smt.ForAll([a,b,c], (a | b) | c == a | (b | c)))
_a, _b,  _c = l.fixes()
#l.unfold(union)
l.rw(ext_ax)
_x = l.fix()
l.rw(elem_union)
l.rw(elem_union)
l.rw(elem_union)
l.rw(elem_union)
l.auto()
union_assoc = l.qed()
#l.rw(bigunion_ax)
#l.rw(bigunion_ax)
#l
#l.split()
#l.intros()
#_B = l.obtain(0)
#l.rw(upair_ax, at=0)
#l.rw(upair_ax)
#l.auto(by=[upair_ax, upair_ax(a,b, _x), upair_ax(b,c, _x), bigunion_ax])
#union_assoc = l.qed().forall([a,b,c])


#l
#l.auto(by=[sep_ax, upair_ax, bigunion_ax])
#inter_distr = l.qed().forall([a,b,c])




#bigunion = smt.Function("bigunion", zf.Set, zf.Set)
#bigunion_ax = kd.axiom()


In [ ]:
kd.Lemma(smt.Implies(A == emp, pick(A)))

In [ ]:
from kdrag.all import *
from kdrag.theories.logic.zf import *

#l.newgoal(elem)

#.intros()
#l
#l.have(smt.ForAll([_x], ))
#l.rw(ext_ax, at=-1)
#l.revert(0)

#l.rw(emp_exists_elem)
#l.simp()
#l.exists(pick(_A))
#l.apply(elem_pick)

#l.auto(by=[elem_pick(_x, _A)])
#l.rw(elem_pick)

In [72]:
l = kd.Lemma(smt.ForAll([a,b,c], a & (b | c) == (a & b) | (a & c)))
l.fixes()
l.rw(ext_ax)
l.fix()
l.rw(elem_inter)
l.rw(elem_union)
l.rw(elem_inter)
l.rw(elem_union)
l.rw(elem_inter)
l.auto()
inter_union_distr = l.qed()


In [ ]:
from kdrag.all import *
from kdrag.theories.logic.zf import *

l = kd.Lemma(smt.ForAll([A], (A == emp) == smt.Not(smt.Exists([x], elem(x, A)))))
_A = l.fix()
l.split()
with l.sub() as l1:
    l1.intros()
    l1.intros()
    _x = l1.obtain(-1)
    l1.show(smt.BoolVal(False), by=[ext_ax, elem_emp])

with l.sub() as l2:
    l2.intros()
    l2.rw(ext_ax)
    _x = l2.fix()
    l2.apply(0)
    l2.exists(_x)
    l2.have(elem(_x, emp) == False, by=[elem_emp(_x)])
    l2.have(elem(_x, _A), by=[])

    l2.auto()
emp_exists_elem = l.qed()
#l.have(smt.ForAll([x], elem(x,A)), by=[ext_ax(A, emp)])


In [ ]:
l = kd.Lemma(smt.ForAll([A], smt.Implies((A != emp), smt.Exists([x], elem(x, A)))))
_A = l.fix()
l.auto(by=[emp_exists_elem(_A)])
l.qed()
not_emp_exists_elem = l.qed()

|= ForAll(A, Implies(A != emp, Exists(x, elem(x, A))))

In [8]:
l = kd.Lemma(smt.ForAll([a,b,c], (a | b) & c == (a & c) | (b & c)))
l.fixes()
l.auto(by=[inter_comm, inter_union_distr])
union_inter_distr = l.qed()
#l.rw(inter_union_distr)
#l.rw(inter_comm)

In [14]:
l = kd.Lemma(smt.ForAll([a,b], a & b <= a))
l.fixes()
l.unfold(le)
l.fix()
l.rw(elem_inter)
l.auto()
inter_le = l.qed()

In [16]:
l = kd.Lemma(smt.ForAll([a,b,c], a <= a | b))
l.fixes()
l.unfold(le)
l.fix()
l.rw(elem_union)
l.auto()
le_union = l.qed()

In [62]:
l = kd.Lemma(smt.ForAll([x,A,B], elem(x, A & B) == (elem(x,A) & elem(x,B))))
l.fixes()
l.unfold(inter)
l.rw(sep_ax)
l.auto()
elem_inter = l.qed()

In [66]:
l = kd.Lemma(smt.ForAll([A,B], A & B == B & A))
l.fixes()
l.rw(ext_ax)
l.fix()
l.rw(elem_inter)
l.rw(elem_inter)
l.auto()
l.qed()

|= ForAll([A, B], inter(A, B) == inter(B, A))

In [ ]:
l = kd.Lemma(smt.ForAll([a,b,c], (a & b) & c == a & (b & c)))
l.fixes()
l.rw(ext_ax)
l.fix()
l.rw(elem_inter)
l.rw(elem_inter)
l.rw(elem_inter)
l.rw(elem_inter)
l.auto()
inter_assoc = l.qed()

|= ForAll([a!0, b!1, c!2],
       inter(inter(a!0, b!1), c!2) ==
       inter(a!0, inter(b!1, c!2)))

In [ ]:

    #_B2 = c2.obtain(0)
    #c2.rw(upair_ax, at=0)
    #c2.auto()

#l.rw(ext_ax)


In [ ]:
elem_union = kd.prove(
    elem(c, union(a, b)) == (elem(c, a) | elem(c, b))),
    by=[elem_bigunion_upair

In [ ]:
l = kd.Lemma((a & b) & c == a & (b & c))
l.unfold(inter)
l.rw(ext_ax)
l.auto(by=[sep_ax])
inter_assoc = l.qed().forall([a,b,c])

l = kd.Lemma(a & b == b & a)
l.unfold(inter)
l.rw(ext_ax)
l.auto(by=[sep_ax])
inter_comm = l.qed().forall([a,b])

In [100]:
l = kd.Lemma(elem(x, a | b) == (elem(x,a) | elem(x,b)))
l.unfold(union)
l.rw(bigunion_ax)
l.split()
l.intros()
_B = l.obtain(0)
l.rw(upair_ax, at=0)
l.auto()

l.intros()
l.cases(elem(x, a))
l.exists(a)
l.rw(upair_ax)
l.auto()
l.exists(b)
l.rw(upair_ax)
l.auto()
l.qed()


|= elem(x, union(a!0, b!1)) == Or(elem(x, a!0), elem(x, b!1))

In [104]:
l = kd.Lemma(elem(x, a & b) == (elem(x,a) & elem(x,b)))
l.unfold(inter)
l.rw(sep_ax)
l.auto()
l.qed()

|= elem(x, inter(a!0, b!1)) == And(elem(x, a!0), elem(x, b!1))

In [ ]:


l = kd.Lemma(a & (b | c) == (a & b) | (a & c))
l.unfold(inter, union)
l.rw(ext_ax)
_x = l.fix()
l.rw(bigunion_ax)
l.rw(sep_ax)
l.simp()
l.rw(bigunion_ax)
l.split()
l.intros()
l.split(at=-1)

_B = l.obtain(0)
l.split(0)
#l.exists(_B)


[x!3390, f!3398(b!1, c!2)];
[elem(f!3398(b!1, c!2), upair(b!1, c!2)),
 elem(x!3390, f!3398(b!1, c!2)),
 elem(x!3390, a!0)]
?|= Exists(B,
       And(elem(B,
                upair(sep(a!0, Lambda(x, elem(x, b!1))),
                      sep(a!0, Lambda(x, elem(x, c!2))))),
           elem(x!3390, B)))

In [ ]:
l = kd.Lemma(a & (b | c) == (a & b) | (a & c))
l.rw(ext_ax)
_x = l.fix()
l.rw(union_ax)
l.rw(inter_ax)
l.auto()

[x!3555];
[]
?|= elem(x!3555, inter(a!0, union(b!1, c!2))) ==
elem(x!3555, union(inter(a!0, b!1), inter(a!0, c!2)))

In [ ]:
l.rw(upair_ax)
l.split()
l.rw(upair_ax, at=0)
l.left()
l.rw(ext_ax)
_x1 = l.fix()
l.rw(sep_ax)
l.apply(-1)
l.rw(ext_ax)
_x2 = l.fix()
l.rw(sep_ax)
l.simp()
#l.auto()
#l.auto(by=[sep_ax])

Uh oh. Is it ok that z3 doesn't know that it needs to skolem though freshvars?

Implicit z3 needs to solve forall(xs, exists(zs, P)). Not(forall, (exists, )) == exists xs, forall za Not(P) is unsat  
But it thinks it's proving exists(xs, exists(zs, P)). Not(exists(xs, exists(zs, P))) ==  forall xs, forall zs, Not(P) is unsat., A stronger statement or is it weaker?


